In [23]:
# spark.stop()

NameError: name 'spark' is not defined

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = SparkConf().setAppName('myApp') \
.setMaster('local')
sc = SparkContext(conf=conf)
spark = SparkSession(sc) 


24/05/22 11:25:17 WARN Utils: Your hostname, Leonardos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.22.17.139 instead (on interface en0)
24/05/22 11:25:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/22 11:25:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.functions import *
import pandas as pd
import math

In [3]:
df_application = spark.read.format('csv').options(header=True,inferSchema=True).load('application_record.csv')
df_application.show(30, truncate=50)
df_application.printSchema()

+-------+-----------+------------+---------------+------------+----------------+--------------------+-----------------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|    NAME_INCOME_TYPE|          NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|
+-------+-----------+------------+---------------+------------+----------------+--------------------+-----------------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|5008804|          M|           Y|              Y|           0|        427500.0|             Working|             Higher education|      Civil marriage| Rented apartment

24/05/22 11:25:29 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
# Check for null etc.
df_application.select([count((when(isnan(c) | isnull(c), 1))).alias(c) for c in df_application.columns]).show()
# No null in data so all data is valid

+---+-----------+------------+---------------+------------+----------------+----------------+-------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
| ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|NAME_INCOME_TYPE|NAME_EDUCATION_TYPE|NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|
+---+-----------+------------+---------------+------------+----------------+----------------+-------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|  0|          0|           0|              0|           0|               0|               0|                  0|                 0|                0|         0|            0|         0|              0|         0|         0|     

In [5]:
# First Check the data in code_gender to check if there is any misdata
# Select and show distinct values for each column separately
df_application.select('code_gender').distinct().show()
df_application.select('flag_own_car').distinct().show()
df_application.select('flag_own_realty').distinct().show()
df_application.select('flag_mobil').distinct().show()
df_application.select('flag_work_phone').distinct().show()
df_application.select('flag_email').distinct().show()
df_application.select('flag_phone').distinct().show()

+-----------+
|code_gender|
+-----------+
|          F|
|          M|
+-----------+

+------------+
|flag_own_car|
+------------+
|           Y|
|           N|
+------------+

+---------------+
|flag_own_realty|
+---------------+
|              Y|
|              N|
+---------------+

+----------+
|flag_mobil|
+----------+
|         1|
+----------+

+---------------+
|flag_work_phone|
+---------------+
|              1|
|              0|
+---------------+

+----------+
|flag_email|
+----------+
|         1|
|         0|
+----------+

+----------+
|flag_phone|
+----------+
|         1|
|         0|
+----------+



In [6]:
# After checking the flag_mobil is not important because the data is only 1 and no other variation. so in the cleaned data not getting the flag mobil
df_app_cleaned = df_application.select(['id','code_gender','flag_own_car','flag_own_realty','cnt_children','amt_income_total',
                                        'name_income_type','name_education_type','name_family_status','name_housing_type','days_birth',
                                        'days_employed','flag_work_phone','flag_phone','flag_email','occupation_type','cnt_fam_members'])

# AND the value is always 0 an 1 as the boolean so we want to make our data uniform so changing the Y as 1 and N as 0, F as 1 and M as 0
df_app_cleaned = df_app_cleaned.withColumn('code_gender', when(df_app_cleaned.code_gender == 'F',1).otherwise(0))
df_app_cleaned = df_app_cleaned.withColumn('flag_own_car', when(df_app_cleaned.flag_own_car == 'Y',1).otherwise(0))
df_app_cleaned = df_app_cleaned.withColumn('flag_own_realty', when(df_app_cleaned.flag_own_car == 'Y',1).otherwise(0))

# More data cleaning cnt_fam_members should be integer because there shouldn't be fam members that is half or less than 1
df_app_cleaned = df_app_cleaned.withColumn('cnt_fam_members', col('cnt_fam_members').cast('Integer'))

df_app_cleaned.show()

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+
|     id|code_gender|flag_own_car|flag_own_realty|cnt_children|amt_income_total|    name_income_type| name_education_type|  name_family_status|name_housing_type|days_birth|days_employed|flag_work_phone|flag_phone|flag_email|occupation_type|cnt_fam_members|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+
|5008804|          0|           1|              0|           0|        427500.0|             Working|    Higher education|      Civil marriage| Rented apartment|    -12005|        -4542|              1|         0|         0|     

In [7]:
# Changing the days birth into age Integer so that it will be classification into its correspondence age
df_app_cleaned = df_app_cleaned.withColumn('age', floor(col('days_birth')/365.25) * -1)

# Check value because age should not be negative or even > 100
df_app_cleaned.select('age').where('age < 0 or age > 100').show()

# Changing the days birth into age Integer so that it will be classification into its correspondence age
df_app_cleaned = df_app_cleaned.withColumn('employed', floor(col('days_employed')/365.25) * -1)

# Check value because age should not be negative or even > 100 as well as the occupation
df_app_cleaned.select(['name_income_type','occupation_type','employed']).where('employed < 0 or employed > 100').distinct().show()

+---+
|age|
+---+
+---+

+----------------+---------------+--------+
|name_income_type|occupation_type|employed|
+----------------+---------------+--------+
|       Pensioner|           NULL|    -999|
+----------------+---------------+--------+



In [8]:
# because the data is said pensioner than it is possible that the employed is negative but for calculation sake it is better to make it into 0
df_app_cleaned = df_app_cleaned.withColumn('employed', when(df_app_cleaned.employed < 0,0).otherwise(df_app_cleaned.employed))

df_app_cleaned.show()

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+---+--------+
|     id|code_gender|flag_own_car|flag_own_realty|cnt_children|amt_income_total|    name_income_type| name_education_type|  name_family_status|name_housing_type|days_birth|days_employed|flag_work_phone|flag_phone|flag_email|occupation_type|cnt_fam_members|age|employed|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+---+--------+
|5008804|          0|           1|              0|           0|        427500.0|             Working|    Higher education|      Civil marriage| Rented apartment|    -12005|        -4542|    

In [9]:
# there is occupation type that is null so we need to fill it up
df_app_cleaned.select('occupation_type').where(isnull(col('occupation_type'))).count() 
# There are no possibility to

134203

In [10]:
# check for occupation type that is null
df_app_cleaned.select(['amt_income_total','name_income_type','occupation_type','employed']).where(isnull(df_app_cleaned.occupation_type)).show()
# there is possibility to use the amt_income_total & name_income_type to classified the occupation_type typically will have the same job
# so need to find the exact number to it

+----------------+----------------+---------------+--------+
|amt_income_total|name_income_type|occupation_type|employed|
+----------------+----------------+---------------+--------+
|        427500.0|         Working|           NULL|      13|
|        427500.0|         Working|           NULL|      13|
|        283500.0|       Pensioner|           NULL|       0|
|        283500.0|       Pensioner|           NULL|       0|
|        283500.0|       Pensioner|           NULL|       0|
|        112500.0|         Working|           NULL|       5|
|        112500.0|         Working|           NULL|       5|
|        112500.0|         Working|           NULL|       5|
|        315000.0|       Pensioner|           NULL|       0|
|        225000.0|         Working|           NULL|       8|
|        225000.0|         Working|           NULL|       8|
|        225000.0|         Working|           NULL|       8|
|        225000.0|         Working|           NULL|       8|
|        157500.0|      

In [11]:
# use amt_income total to get the kind of occupation type check first
# the possibility is that the amt_income_total,name_income_type, and employed can fill the occupation_type that is null
df_app_cleaned.select(['amt_income_total','name_income_type','occupation_type','employed']).where((df_app_cleaned.amt_income_total == 225000.0) & (df_app_cleaned.name_income_type == 'Working')).show()
# This data further proven the point that it will be able to be filled up

+----------------+----------------+---------------+--------+
|amt_income_total|name_income_type|occupation_type|employed|
+----------------+----------------+---------------+--------+
|        225000.0|         Working|           NULL|       8|
|        225000.0|         Working|           NULL|       8|
|        225000.0|         Working|           NULL|       8|
|        225000.0|         Working|           NULL|       8|
|        225000.0|         Working|       Managers|       3|
|        225000.0|         Working|       Managers|       3|
|        225000.0|         Working|       Managers|       3|
|        225000.0|         Working|       Managers|       3|
|        225000.0|         Working|       Managers|       3|
|        225000.0|         Working|       Managers|       3|
|        225000.0|         Working|       Managers|       3|
|        225000.0|         Working|       Managers|       3|
|        225000.0|         Working|       Managers|       3|
|        225000.0|      

In [12]:
# so use the possibility of it and fill the occupation_type that got from the amt_income_total and name_income_type
from pyspark.sql import Window, functions as F

# Define a window specification
window_spec = Window.partitionBy('amt_income_total', 'name_income_type')

# Fill missing values of occupation_type with the most frequent value within each group
filled_occupation_type = F.first('occupation_type', ignorenulls=True).over(window_spec)

# Fill missing values using coalesce
df_app_cleaned = df_app_cleaned.withColumn('filled_occupation_type', filled_occupation_type)
df_app_cleaned = df_app_cleaned.withColumn('occupation_type', F.coalesce('occupation_type', 'filled_occupation_type')).drop('filled_occupation_type')

df_app_cleaned.show()

+-------+-----------+------------+---------------+------------+----------------+----------------+--------------------+--------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+---+--------+
|     id|code_gender|flag_own_car|flag_own_realty|cnt_children|amt_income_total|name_income_type| name_education_type|  name_family_status|name_housing_type|days_birth|days_employed|flag_work_phone|flag_phone|flag_email|occupation_type|cnt_fam_members|age|employed|
+-------+-----------+------------+---------------+------------+----------------+----------------+--------------------+--------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+---+--------+
|6093713|          1|           0|              0|           0|         26100.0|       Pensioner|Secondary / secon...|             Married|House / apartment|    -21003|       365243|              0|    

In [13]:
# check for occupation type that is null  and that the amt_income_total and name_income_type same, but still null
df_app_cleaned.select(['amt_income_total','name_income_type','occupation_type','employed']).where(isnull(df_app_cleaned.occupation_type) & (df_app_cleaned.employed != 0)).show()
# There are no exact data to fill the null in that missing value, so the possibility is the same name_income_type with the nearest amt_income_total
# Create a window specification to partition by 'name_income_type' and order by the absolute difference in 'amt_income_total'
window_spec = Window.partitionBy('name_income_type').orderBy(F.abs(F.col('amt_income_total') - F.col('amt_income_total_with_null')))

# Create a temporary column to store the amt_income_total of rows with null occupation_type
df_temp = df_app_cleaned.withColumn('amt_income_total_with_null', F.when(F.col('occupation_type').isNull(), F.col('amt_income_total')))

# Fill missing occupation_type using the nearest amt_income_total
filled_occupation_type = F.first('occupation_type', ignorenulls=True).over(window_spec)
df_temp = df_temp.withColumn('filled_occupation_type', filled_occupation_type)

# Use coalesce to fill the null values
df_app_cleaned = df_temp.withColumn('occupation_type', F.coalesce(F.col('occupation_type'), F.col('filled_occupation_type'))).drop('filled_occupation_type', 'amt_income_total_with_null')

df_app_cleaned.show()

+----------------+--------------------+---------------+--------+
|amt_income_total|    name_income_type|occupation_type|employed|
+----------------+--------------------+---------------+--------+
|         32400.0|             Working|           NULL|       5|
|         32400.0|             Working|           NULL|       5|
|         32400.0|             Working|           NULL|       5|
|         38250.0|       State servant|           NULL|       3|
|         38250.0|       State servant|           NULL|       3|
|         38250.0|       State servant|           NULL|       3|
|         38250.0|       State servant|           NULL|       3|
|         38250.0|       State servant|           NULL|       3|
|         38250.0|       State servant|           NULL|       3|
|         38250.0|       State servant|           NULL|       3|
|         38700.0|             Working|           NULL|      26|
|         41211.0|Commercial associate|           NULL|       6|
|         41211.0|Commerc

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+---+--------+
|     id|code_gender|flag_own_car|flag_own_realty|cnt_children|amt_income_total|    name_income_type| name_education_type|name_family_status|name_housing_type|days_birth|days_employed|flag_work_phone|flag_phone|flag_email|occupation_type|cnt_fam_members|age|employed|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+---+--------+
|6014233|          1|           0|              0|           1|         28723.5|Commercial associate|Secondary / secon...|           Married|House / apartment|    -14361|         -138|            

In [14]:
# check for occupation type that is null
df_app_cleaned.select(['amt_income_total','name_income_type','occupation_type','employed']).where(isnull(df_app_cleaned.occupation_type)).show()
# Data already cleaned!!

+----------------+----------------+---------------+--------+
|amt_income_total|name_income_type|occupation_type|employed|
+----------------+----------------+---------------+--------+
+----------------+----------------+---------------+--------+



In [25]:
df_credit = spark.read.format('csv').options(header=True,inferSchema=True).load('credit_record.csv')
df_credit.show(30, truncate=50)
df_credit.printSchema()
print('Status \n\
      0 => 1-29 days past due \n\
      1 => 30-59 days past due \n\
      2 => 60-89 days past due \n\
      3 => 90-119 days past due \n\
      4 => 120-149 days past due \n\
      5 => overdue/ bad debts > 150 days \n\
      C => paid successfully \n\
      X => No loan \n\
')

+-------+--------------+------+
|     ID|MONTHS_BALANCE|STATUS|
+-------+--------------+------+
|5001711|             0|     X|
|5001711|            -1|     0|
|5001711|            -2|     0|
|5001711|            -3|     0|
|5001712|             0|     C|
|5001712|            -1|     C|
|5001712|            -2|     C|
|5001712|            -3|     C|
|5001712|            -4|     C|
|5001712|            -5|     C|
|5001712|            -6|     C|
|5001712|            -7|     C|
|5001712|            -8|     C|
|5001712|            -9|     0|
|5001712|           -10|     0|
|5001712|           -11|     0|
|5001712|           -12|     0|
|5001712|           -13|     0|
|5001712|           -14|     0|
|5001712|           -15|     0|
|5001712|           -16|     0|
|5001712|           -17|     0|
|5001712|           -18|     0|
|5001713|             0|     X|
|5001713|            -1|     X|
|5001713|            -2|     X|
|5001713|            -3|     X|
|5001713|            -4|     X|
|5001713

In [26]:
# Check for null etc.
df_credit.select([count((when(isnan(c) | isnull(c), 1))).alias(c) for c in df_credit.columns]).show()
# No null in data so all data is valid

+---+--------------+------+
| ID|MONTHS_BALANCE|STATUS|
+---+--------------+------+
|  0|             0|     0|
+---+--------------+------+

